In [12]:
import sys
import pandas as pd
import json
import numpy as np
import torch
from torch import Tensor
import pickle
from tqdm.auto import tqdm

from matplotlib import pyplot as plt

sys.path.append("../")
from src.log import myLogger
from src.repository.data_repository import DataRepository
from src.checkpoint.checkpoint import Checkpoint
from src.metrics.jaccard import jaccard

pd.set_option("display.max_columns", 300)
pd.set_option("display.max_rows", 600)

%load_ext autoreload
%autoreload 2

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))
%config InlineBackend.figure_formats = {'png', 'retina'}

In [2]:
logger = myLogger("../logs/temp.log", exp_id="e000", wdb_prj_id="temp", exp_config=None, use_wdb=False) 
dr = DataRepository(logger=logger, local_root_path="..")

2021-11-14 13:12:09,004 log.py               31   [INFO] [__init__] skip wandb init 


In [3]:
import pickle

train_prep_df = dr.load_preprocessed_df(
          dataset_name="train",                                                                                                                                                                                                                                   
          class_name="BaselineKernelPreprocessorV1",
          tokenizer_name="XLMRobertaTokenizerFast",
          max_length=400,
          pad_on_right=True,
          stride=135,
          split=False,
          lstrip=False,
          use_language_as_question=False,
          add_overflowing_batch_id=False,
    )

mlqa_hindi_prep_df = dr.load_preprocessed_df(
          dataset_name="mlqa_hindi",                                                                                                                                                                                                                                   
          class_name="BaselineKernelPreprocessorV1",
          tokenizer_name="XLMRobertaTokenizerFast",
          max_length=400,
          pad_on_right=True,
          stride=135,
          split=False,
          lstrip=False,
          use_language_as_question=False,
          add_overflowing_batch_id=False,
    )

xquad_prep_df = dr.load_preprocessed_df(
          dataset_name="xquad",                                                                                                                                                                                                                                   
          class_name="BaselineKernelPreprocessorV1",
          tokenizer_name="XLMRobertaTokenizerFast",
          max_length=400,
          pad_on_right=True,
          stride=135,
          split=False,
          lstrip=False,
          use_language_as_question=False,
          add_overflowing_batch_id=False,
    )

prep_df = pd.concat([train_prep_df, mlqa_hindi_prep_df, xquad_prep_df], axis=0).reset_index(drop=True)
prep_df

2021-11-14 13:12:09,412 data_repository.py   262  [INFO] [load_preprocessed_df] loading data/preprocessed/train_BaselineKernelPreprocessorV1_XLMRobertaTokenizerFast_400_True_135_False_False_False_False.pkl ... 
2021-11-14 13:12:12,534 data_repository.py   269  [INFO] [load_preprocessed_df] done. 
2021-11-14 13:12:12,535 data_repository.py   262  [INFO] [load_preprocessed_df] loading data/preprocessed/mlqa_hindi_BaselineKernelPreprocessorV1_XLMRobertaTokenizerFast_400_True_135_False_False_False_False.pkl ... 
2021-11-14 13:12:14,197 data_repository.py   269  [INFO] [load_preprocessed_df] done. 
2021-11-14 13:12:14,199 data_repository.py   262  [INFO] [load_preprocessed_df] loading data/preprocessed/xquad_BaselineKernelPreprocessorV1_XLMRobertaTokenizerFast_400_True_135_False_False_False_False.pkl ... 
2021-11-14 13:12:14,407 data_repository.py   269  [INFO] [load_preprocessed_df] done. 


,id,context,question,answer_text,answer_start,language,top20_context,answer_text_count,input_ids,token_type_ids,attention_mask,sequence_ids,offset_mapping,overflowing_batch_id,duplicated_elems_num_with,part_answer_text_count,start_position,end_position,segmentation_position,is_contain_answer_text
0,903deec17,ஒரு சாதாரண வளர்ந்த மனிதனுடைய எலும்புக்கூடு பின...,மனித உடலில் எத்தனை எலும்புகள் உள்ளன?,206,53,tamil,ஒரு சாதாரண வளர்ந்த ம,1,"[0, 69535, 81049, 37368, 153264, 12095, 52989,...","[None, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, None, Non...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[(-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, ...",0,-1,1,27,27,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1
1,903deec17,ஒரு சாதாரண வளர்ந்த மனிதனுடைய எலும்புக்கூடு பின...,மனித உடலில் எத்தனை எலும்புகள் உள்ளன?,206,53,tamil,ஒரு சாதாரண வளர்ந்த ம,1,"[0, 69535, 81049, 37368, 153264, 12095, 52989,...","[None, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, None, Non...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[(-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, ...",1,-1,0,0,0,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0
2,903deec17,ஒரு சாதாரண வளர்ந்த மனிதனுடைய எலும்புக்கூடு பின...,மனித உடலில் எத்தனை எலும்புகள் உள்ளன?,206,53,tamil,ஒரு சாதாரண வளர்ந்த ம,1,"[0, 69535, 81049, 37368, 153264, 12095, 52989,...","[None, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, None, Non...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[(-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, ...",2,-1,0,0,0,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0
3,903deec17,ஒரு சாதாரண வளர்ந்த மனிதனுடைய எலும்புக்கூடு பின...,மனித உடலில் எத்தனை எலும்புகள் உள்ளன?,206,53,tamil,ஒரு சாதாரண வளர்ந்த ம,1,"[0, 69535, 81049, 37368, 153264, 12095, 52989,...","[None, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, None, Non...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[(-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, ...",3,-1,0,0,0,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0
4,903deec17,ஒரு சாதாரண வளர்ந்த மனிதனுடைய எலும்புக்கூடு பின...,மனித உடலில் எத்தனை எலும்புகள் உள்ளன?,206,53,tamil,ஒரு சாதாரண வளர்ந்த ம,1,"[0, 69535, 81049, 37368, 153264, 12095, 52989,...","[None, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, None, Non...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[(-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, ...",4,-1,0,0,0,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22321,xquad_1185,विद्युत आवेश के परिवर्तन की समय दर के रूप में ...,इलेक्ट्रोस्टैटिक और चुंबकीय बल के योग के रूप क...,इलेक्ट्रोस्टैटिक बल,328,hindi,विद्युत आवेश के परिव,1,"[0, 234186, 2284, 17433, 3045, 71683, 871, 369...","[None, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[(-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, ...",0,-1,1,116,121,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1
22322,xquad_1186,उस आयतन के लिए प्रासंगिक क्रॉस-सेक्शनल क्षेत्र...,संरचनाओं में तनाव का कारण क्या बनता है?,तनाव टेंसर,343,hindi,उस आयतन के लिए प्रास,1,"[0, 120018, 11846, 421, 129558, 641, 6701, 600...","[None, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, None, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[(-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, ...",0,-1,1,128,131,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1
22323,xquad_1187,उस आयतन के लिए प्रासंगिक क्रॉस-सेक्शनल क्षेत्र...,किसी वस्तु के आयतन में क्रॉस सेक्शन क्षेत्र की...,दबाव की शर्तें,118,hindi,उस आयतन के लिए प्रास,1,"[0, 12820, 89773, 287, 34889, 41420, 421, 4761...","[None, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 1, 2, 3, 4,

#### load values

In [5]:
ids = []
answer_preds = []
answer_texts = []
# jaccards = [jaccard(answer_pred, answer_text) for answer_pred, answer_text in zip(answer_preds, answer_texts)]

In [6]:
import gc
from collections import defaultdict
from tqdm.auto import tqdm

target_exp_ids = ["e072"]

checkpoints_info = {}

for target_exp_id in target_exp_ids:
    checkpoints_info[target_exp_id] = defaultdict(list)
    best_checkpoints = dr.best_checkpoint_filepaths(target_exp_id)
    for best_checkpoint in tqdm(best_checkpoints):
        exp_fold_checkpoint = dr.load_checkpoint_from_filepath(filepath_from_root=best_checkpoint, load_from_gcs=True, rm_local_after_load=True) 
        del exp_fold_checkpoint.model_state_dict
        del exp_fold_checkpoint.scheduler_state_dict
        del exp_fold_checkpoint.optimizer_state_dict
        gc.collect()
        
        ids.extend(exp_fold_checkpoint.val_pospro_ids)
        answer_preds.extend(exp_fold_checkpoint.val_pospro_answer_preds)
        answer_texts.extend(exp_fold_checkpoint.val_pospro_answer_texts)
        
        checkpoints_info[target_exp_id]["val_ids"].extend(exp_fold_checkpoint.val_ids)
        checkpoints_info[target_exp_id]["val_start_logits"].extend(exp_fold_checkpoint.val_start_logits)
        checkpoints_info[target_exp_id]["val_end_logits"].extend(exp_fold_checkpoint.val_end_logits)
        del exp_fold_checkpoint
        gc.collect()
    break

  0%|          | 0/5 [00:00<?, ?it/s]

2021-11-14 13:20:33,838 repository.py        167  [INFO] [__download_from_gcs] downloading data/checkpoint/e072/best_checkpoint/0_1_1.3986_0.6728.pkl from gs://kaggle-chaii-2021/../data/checkpoint/e072/best_checkpoint/0_1_1.3986_0.6728.pkl 
2021-11-14 13:21:27,166 repository.py        176  [INFO] [__download_from_gcs] download done. 
2021-11-14 13:21:41,342 repository.py        167  [INFO] [__download_from_gcs] downloading data/checkpoint/e072/best_checkpoint/1_2_1.3330_0.6974.pkl from gs://kaggle-chaii-2021/../data/checkpoint/e072/best_checkpoint/1_2_1.3330_0.6974.pkl 
2021-11-14 13:22:36,996 repository.py        176  [INFO] [__download_from_gcs] download done. 
2021-11-14 13:24:23,442 repository.py        167  [INFO] [__download_from_gcs] downloading data/checkpoint/e072/best_checkpoint/2_1_1.6197_0.6928.pkl from gs://kaggle-chaii-2021/../data/checkpoint/e072/best_checkpoint/2_1_1.6197_0.6928.pkl 
2021-11-14 13:25:19,463 repository.py        176  [INFO] [__download_from_gcs] download

In [33]:
import Levenshtein

jaccards = [jaccard(answer_pred, answer_text) for answer_pred, answer_text in zip(answer_preds, answer_texts)]
levenshteins = [Levenshtein.distance(" ".join(answer_pred.split()), " ".join(answer_text.split())) for answer_pred, answer_text in zip(answer_preds, answer_texts)]

answer_pred_text_df = pd.DataFrame()
answer_pred_text_df["id"] = ids
answer_pred_text_df["answer_pred"] = answer_preds
answer_pred_text_df["answer_text"] = answer_texts
answer_pred_text_df["jaccard"] = jaccards
answer_pred_text_df["levenshtein"] = levenshteins
answer_pred_text_df

,id,answer_pred,answer_text,jaccard,levenshtein
0,003332744,தமிழ் மொழி,தமிழ்,0.500000,5
1,01c9df949,भारत,भारत,1.000000,0
2,02c482c70,अरस्तू,अरस्तू,1.000000,0
3,03d39d040,1989,1909,0.000000,1
4,047e3b82d,1882,19 मार्च 1882,0.333333,9
...,...,...,...,...,...
7724,xquad_979,0.3 से 0.6,0.3 से 0.6 तक,0.750000,3
7725,xquad_980,तारीख सही नहीं है,तारीख,0.250000,12
7726,xquad_981,तारीख सही नहीं है। उन्होंने इस उदाहरण में आईप...,इस उदाहरण में आईपीसीसी की भलीभांति स्थापित प्र...,0.687500,28
7727,xquad_982,आईसीएसआई रिपोर्ट,डब्ल्यूडब्ल्यूएफ़ की रिपोर्ट,0.250000,18


#### merge values

In [11]:
checkpoints_info.keys()

dict_keys(['e072'])

In [12]:
# checkpoints_info["e049"]["val_ids"] == checkpoints_info["e059"]["val_ids"]

In [13]:
for target_exp_id in checkpoints_info.keys():
    checkpoints_info[target_exp_id]["val_start_logits"] = np.asarray(checkpoints_info[target_exp_id]["val_start_logits"])
    checkpoints_info[target_exp_id]["val_end_logits"] = np.asarray(checkpoints_info[target_exp_id]["val_end_logits"])
    
# for target_exp_id in checkpoints_info.keys():
#     checkpoints_info[target_exp_id]["start_logits"] = [np.asarray(logit) for logit in checkpoints_info[target_exp_id]["start_logits"]]
#     checkpoints_info[target_exp_id]["end_logits"] = [np.asarray(logit) for logit in checkpoints_info[target_exp_id]["end_logits"]]

In [14]:
ensembled_checkpoint_info = {
    "val_ids": None,
    "val_start_logits": None,
    "val_end_logits": None,
}

for target_exp_id in checkpoints_info.keys():
    if ensembled_checkpoint_info["val_ids"] is None:
        ensembled_checkpoint_info["val_ids"] = checkpoints_info[target_exp_id]["val_ids"]
    if ensembled_checkpoint_info["val_start_logits"] is None:
        ensembled_checkpoint_info["val_start_logits"] = checkpoints_info[target_exp_id]["val_start_logits"]
    else:
        ensembled_checkpoint_info["val_start_logits"] += checkpoints_info[target_exp_id]["val_start_logits"]
    if ensembled_checkpoint_info["val_end_logits"] is None:
        ensembled_checkpoint_info["val_end_logits"] = checkpoints_info[target_exp_id]["val_end_logits"]
    else:
        ensembled_checkpoint_info["val_end_logits"] += checkpoints_info[target_exp_id]["val_end_logits"]
        
        
ensembled_checkpoint_info["val_start_logits"] = [np.asarray(logit) for logit in checkpoints_info[target_exp_id]["val_start_logits"]]
ensembled_checkpoint_info["val_end_logits"] = [np.asarray(logit) for logit in checkpoints_info[target_exp_id]["val_end_logits"]]

In [15]:
checkpoint_df = pd.DataFrame()
checkpoint_df["id"] = ensembled_checkpoint_info["val_ids"]
checkpoint_df["start_logits"] = ensembled_checkpoint_info["val_start_logits"]
checkpoint_df["end_logits"] = ensembled_checkpoint_info["val_end_logits"]

checkpoint_df["overflowing_batch_id"] = None
bef_id = ""
overflowing_batch_id = 0
for i, row in checkpoint_df.iterrows():
    if str(row["id"]) != bef_id:
        overflowing_batch_id = 0
    checkpoint_df.loc[i, "overflowing_batch_id"] = overflowing_batch_id
    bef_id = row["id"]
    overflowing_batch_id += 1
checkpoint_df.head(10)

,id,start_logits,end_logits,overflowing_batch_id
0,903deec17,"[4.1968607902526855, -8.74495792388916, -9.496...","[1.6140170097351074, -11.599809646606445, -11....",0
1,903deec17,"[6.063086032867432, -8.200494766235352, -9.124...","[4.183827877044678, -11.644315719604492, -11.5...",1
2,903deec17,"[6.470153331756592, -8.224721908569336, -9.086...","[4.527822017669678, -11.55895709991455, -11.42...",2
3,903deec17,"[5.89698600769043, -8.526455879211426, -9.3145...","[4.094972610473633, -11.802474975585938, -11.7...",3
4,903deec17,"[6.092099189758301, -7.509745121002197, -9.079...","[4.20054292678833, -11.432998657226562, -11.35...",4
5,903deec17,"[5.858525276184082, -7.377020835876465, -8.979...","[4.1266255378723145, -11.359797477722168, -11....",5
6,29d154b56,"[5.148509502410889, -9.699833869934082, -10.26...","[2.5000622272491455, -12.07172966003418, -11.5...",0
7,29d154b56,"[5.783593654632568, -9.623424530029297, -10.40...","[3.5271825790405273, -12.145627975463867, -11....",1
8,29d154b56,"[6.385714054107666, -9.0951566696167, -10.0352...","[4.595269203186035, -12.098862648010254, -11.2...",2
9,29d154b56,"[6.146578788757324, -9.62275505065918, -10.109...","[4.3153300285339355, -11.996927261352539, -11....",3


In [16]:
prep_df = prep_df.merge(checkpoint_df, on=["id", "overflowing_batch_id"], how="left")
prep_df

,id,context,question,answer_text,answer_start,language,top20_context,answer_text_count,input_ids,token_type_ids,attention_mask,sequence_ids,offset_mapping,overflowing_batch_id,duplicated_elems_num_with,part_answer_text_count,start_position,end_position,segmentation_position,is_contain_answer_text,start_logits,end_logits
0,903deec17,ஒரு சாதாரண வளர்ந்த மனிதனுடைய எலும்புக்கூடு பின...,மனித உடலில் எத்தனை எலும்புகள் உள்ளன?,206,53,tamil,ஒரு சாதாரண வளர்ந்த ம,1,"[0, 69535, 81049, 37368, 153264, 12095, 52989,...","[None, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, None, Non...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[(-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, ...",0,-1,1,27,27,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1,"[4.1968607902526855, -8.74495792388916, -9.496...","[1.6140170097351074, -11.599809646606445, -11...."
1,903deec17,ஒரு சாதாரண வளர்ந்த மனிதனுடைய எலும்புக்கூடு பின...,மனித உடலில் எத்தனை எலும்புகள் உள்ளன?,206,53,tamil,ஒரு சாதாரண வளர்ந்த ம,1,"[0, 69535, 81049, 37368, 153264, 12095, 52989,...","[None, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, None, Non...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[(-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, ...",1,-1,0,0,0,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0,"[6.063086032867432, -8.200494766235352, -9.124...","[4.183827877044678, -11.644315719604492, -11.5..."
2,903deec17,ஒரு சாதாரண வளர்ந்த மனிதனுடைய எலும்புக்கூடு பின...,மனித உடலில் எத்தனை எலும்புகள் உள்ளன?,206,53,tamil,ஒரு சாதாரண வளர்ந்த ம,1,"[0, 69535, 81049, 37368, 153264, 12095, 52989,...","[None, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, None, Non...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[(-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, ...",2,-1,0,0,0,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0,"[6.470153331756592, -8.224721908569336, -9.086...","[4.527822017669678, -11.55895709991455, -11.42..."
3,903deec17,ஒரு சாதாரண வளர்ந்த மனிதனுடைய எலும்புக்கூடு பின...,மனித உடலில் எத்தனை எலும்புகள் உள்ளன?,206,53,tamil,ஒரு சாதாரண வளர்ந்த ம,1,"[0, 69535, 81049, 37368, 153264, 12095, 52989,...","[None, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, None, Non...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[(-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, ...",3,-1,0,0,0,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0,"[5.89698600769043, -8.526455879211426, -9.3145...","[4.094972610473633, -11.802474975585938, -11.7..."
4,903deec17,ஒரு சாதாரண வளர்ந்த மனிதனுடைய எலும்புக்கூடு பின...,மனித உடலில் எத்தனை எலும்புகள் உள்ளன?,206,53,tamil,ஒரு சாதாரண வளர்ந்த ம,1,"[0, 69535, 81049, 37368, 153264, 12095, 52989,...","[None, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, None, Non...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[(-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, ...",4,-1,0,0,0,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0,"[6.092099189758301, -7.509745121002197, -9.079...","[4.20054292678833, -11.432998657226562, -11.35..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22321,xquad_1185,विद्युत आवेश के परिवर्तन की समय दर के रूप में ...,इलेक्ट्रोस्टैटिक और चुंबकीय बल के योग के रूप क...,इलेक्ट्रोस्टैटिक बल,328,hindi,विद्युत आवेश के परिव,1,"[0, 234186, 2284, 17433, 3045, 71683, 871, 369...","[None, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[(-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, ...",0,-1,1,116,121,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1,"[3.3736329078674316, -5.083080768585205, -8.84...","[0.5784845948219299, -10.55335521697998, -11.0..."
22322,xquad_1186,उस आयतन के लिए प्रासंगिक क्रॉस-सेक्शनल क्षेत्र...,संरचनाओं में तनाव का कारण क्या बनता है?,तनाव टेंसर,343,hindi,उस आयतन के लिए प्रास,1,"[0, 12001

In [17]:
from typing import List, Tuple

def get_context_part(
    offset_mapping: List[Tuple[int, int]], context: str
) -> str:
    s = 1_000_000_000
    e = 0
    for (offs, offe) in offset_mapping:
        if offs == -1:
            continue
        s = min(offs, s)
        e = max(offe, e)
    context_part = context[s:e]
    return s, e, context_part

def get_part_start_end_logit_score(start_char_index, end_char_index, offset_mapping, start_logit, end_logit):
    start_index = 0
    while offset_mapping[start_index][0] == -1:
        start_index += 1
    while offset_mapping[start_index][0] != -1 and offset_mapping[start_index][0] <= start_char_index:
        start_index += 1
    start_index -= 1
    end_index = len(offset_mapping) - 1
    while offset_mapping[end_index][1] == -1:
        end_index -= 1
    while offset_mapping[end_index][1] != -1 and offset_mapping[end_index][1] >= end_char_index:
        end_index -= 1
    end_index += 1
    score = start_logit[start_index] + end_logit[end_index]
    return score

In [18]:
# %debug
import json

for i, row in tqdm(prep_df.iterrows(), total=len(prep_df)):
    context_part_start_char_index, _, context_part = get_context_part(row["offset_mapping"], row["context"])
    char_index_score_dict = {}
    start_char_index = context_part.find(row["answer_text"])
    while start_char_index >= 0:
        score = get_part_start_end_logit_score(
            context_part_start_char_index+start_char_index,
            context_part_start_char_index+start_char_index+len(row["answer_text"]),
            row["offset_mapping"],
            row["start_logits"],
            row["end_logits"]
        )
        char_index_score_dict[context_part_start_char_index+start_char_index] = score

        context_part_start_char_index += (start_char_index+len(row["answer_text"]))
        context_part = context_part[start_char_index+len(row["answer_text"]):]
        start_char_index = context_part.find(row["answer_text"])
    prep_df.loc[i, "char_index_score_dict"] = json.dumps(char_index_score_dict)

  0%|          | 0/22326 [00:00<?, ?it/s]

In [19]:
prep_df.head(100)[["id", "question", "answer_text", "answer_start", "language", "overflowing_batch_id", "is_contain_answer_text", "char_index_score_dict"]]

,id,question,answer_text,answer_start,language,overflowing_batch_id,is_contain_answer_text,char_index_score_dict
0,903deec17,மனித உடலில் எத்தனை எலும்புகள் உள்ளன?,206,53,tamil,0,1,"{""53"": 12.486751556396484}"
1,903deec17,மனித உடலில் எத்தனை எலும்புகள் உள்ளன?,206,53,tamil,1,0,{}
2,903deec17,மனித உடலில் எத்தனை எலும்புகள் உள்ளன?,206,53,tamil,2,0,{}
3,903deec17,மனித உடலில் எத்தனை எலும்புகள் உள்ளன?,206,53,tamil,3,0,{}
4,903deec17,மனித உடலில் எத்தனை எலும்புகள் உள்ளன?,206,53,tamil,4,0,{}
5,903deec17,மனித உடலில் எத்தனை எலும்புகள் உள்ளன?,206,53,tamil,5,0,{}
6,d9841668c,காளிதாசன் எங்கு பிறந்தார்?,காசுமீரில்,2358,tamil,0,0,{}
7,d9841668c,காளிதாசன் எங்கு பிறந்தார்?,காசுமீரில்,2358,tamil,1,0,{}
8,d9841668c,காளிதாசன் எங்கு பிறந்தார்?,காசுமீரில்,2358,tamil,2,1,"{""2358"": 3.1008405089378357, ""2531"": -15.44681..."
9,d9841668c,காளிதாசன் எங்கு பிறந்தார்?,காசுமீரில்,2358,tamil,3,0,"{""2653"": -19.440195083618164}"


In [20]:
def temp(x):
    y = json.loads(x)
    if len(y) == 0:
        return -100000000
    else:
        return max(y.values())
    
def temp2(x):
    y = json.loads(x)
    if len(y) == 0:
        return -100000000
    else:
        res_k = -1
        res_v = -1_000_000_000
        for k, v in y.items():
            if v > res_v:
                res_k = int(k)
                res_v = v
        return res_k
    
prep_df["max_score"] = prep_df.char_index_score_dict.apply(lambda x: temp(x))
prep_df["max_index"] = prep_df.char_index_score_dict.apply(lambda x: temp2(x))
prep_df

,id,context,question,answer_text,answer_start,language,top20_context,answer_text_count,input_ids,token_type_ids,attention_mask,sequence_ids,offset_mapping,overflowing_batch_id,duplicated_elems_num_with,part_answer_text_count,start_position,end_position,segmentation_position,is_contain_answer_text,start_logits,end_logits,char_index_score_dict,max_score,max_index
0,903deec17,ஒரு சாதாரண வளர்ந்த மனிதனுடைய எலும்புக்கூடு பின...,மனித உடலில் எத்தனை எலும்புகள் உள்ளன?,206,53,tamil,ஒரு சாதாரண வளர்ந்த ம,1,"[0, 69535, 81049, 37368, 153264, 12095, 52989,...","[None, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, None, Non...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[(-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, ...",0,-1,1,27,27,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1,"[4.1968607902526855, -8.74495792388916, -9.496...","[1.6140170097351074, -11.599809646606445, -11....","{""53"": 12.486751556396484}",1.248675e+01,53
1,903deec17,ஒரு சாதாரண வளர்ந்த மனிதனுடைய எலும்புக்கூடு பின...,மனித உடலில் எத்தனை எலும்புகள் உள்ளன?,206,53,tamil,ஒரு சாதாரண வளர்ந்த ம,1,"[0, 69535, 81049, 37368, 153264, 12095, 52989,...","[None, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, None, Non...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[(-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, ...",1,-1,0,0,0,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0,"[6.063086032867432, -8.200494766235352, -9.124...","[4.183827877044678, -11.644315719604492, -11.5...",{},-1.000000e+08,-100000000
2,903deec17,ஒரு சாதாரண வளர்ந்த மனிதனுடைய எலும்புக்கூடு பின...,மனித உடலில் எத்தனை எலும்புகள் உள்ளன?,206,53,tamil,ஒரு சாதாரண வளர்ந்த ம,1,"[0, 69535, 81049, 37368, 153264, 12095, 52989,...","[None, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, None, Non...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[(-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, ...",2,-1,0,0,0,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0,"[6.470153331756592, -8.224721908569336, -9.086...","[4.527822017669678, -11.55895709991455, -11.42...",{},-1.000000e+08,-100000000
3,903deec17,ஒரு சாதாரண வளர்ந்த மனிதனுடைய எலும்புக்கூடு பின...,மனித உடலில் எத்தனை எலும்புகள் உள்ளன?,206,53,tamil,ஒரு சாதாரண வளர்ந்த ம,1,"[0, 69535, 81049, 37368, 153264, 12095, 52989,...","[None, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, None, Non...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[(-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, ...",3,-1,0,0,0,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0,"[5.89698600769043, -8.526455879211426, -9.3145...","[4.094972610473633, -11.802474975585938, -11.7...",{},-1.000000e+08,-100000000
4,903deec17,ஒரு சாதாரண வளர்ந்த மனிதனுடைய எலும்புக்கூடு பின...,மனித உடலில் எத்தனை எலும்புகள் உள்ளன?,206,53,tamil,ஒரு சாதாரண வளர்ந்த ம,1,"[0, 69535, 81049, 37368, 153264, 12095, 52989,...","[None, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, None, Non...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[(-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, ...",4,-1,0,0,0,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0,"[6.092099189758301, -7.509745121002197, -9.079...","[4.20054292678833, -11.432998657226562, -11.35...",{},-1.000000e+08,-100000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22321,xquad_1185,विद्युत आवेश के परिवर्तन की समय दर के रूप में ...,इलेक्ट्रोस्टैटिक और चुंबकीय बल के योग के रूप क...,इलेक्ट्रोस्टैटिक बल,328,hindi,विद्युत आवेश के परिव,1,"[0, 234186, 2284, 17433, 3045, 71683, 871, 369...","[None, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[(-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, ...",0,-1,1,116,121,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1,"[3.3736329078674316, -5.083080768585205, -8.84...",

In [179]:
row = prep_df.loc[10227]
print(row["is_contain_answer_text"])
print(row["question"])
print(row["answer_text"])
print(row["char_index_score_dict"])
print(row["offset_mapping"])
print(row["context"][5014:6244])

1
एंड्रयू हीथ लेजर ने सबसे पहले किस फिल्म में काम किया था?
ब्लैकरोच
{"5792": 0.13439003378152847}
[(-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, -1), (5014, 5017), (5017, 5020), (5020, 5023), (5023, 5029), (5029, 5033), (5033, 5040), (5040, 5042), (5042, 5043), (5043, 5045), (5045, 5048), (5048, 5054), (5054, 5055), (5055, 5057), (5057, 5067), (5067, 5070), (5070, 5073), (5073, 5074), (5074, 5077), (5077, 5079), (5079, 5080), (5080, 5083), (5083, 5086), (5086, 5090), (5090, 5095), (5095, 5096), (5096, 5097), (5097, 5101), (5101, 5105), (5105, 5109), (5109, 5113), (5114, 5120), (5122, 5127), (5127, 5130), (5130, 5134), (5135, 5138), (5138, 5143), (5143, 5146), (5146, 5150), (5150, 5154), (5154, 5161), (5161, 5167), (5167, 5170), (5170, 5178), (5178, 5183), (5183, 5186), (5186, 5190), (5190, 5193), (5193, 5195), (5195, 5198), (5198, 5204), (

In [34]:
answer_pred_text_df = answer_pred_text_df.merge(prep_df.drop_duplicates("id")[["id", "question", "language", "context"]], on="id", how="left")

In [37]:
with open("../data/preprocessed/e072_answer_pred_text_df .pkl", "wb") as fout:
    pickle.dump(answer_pred_text_df, fout)

## どんな間違え方をしてる？

In [13]:
with open("../data/preprocessed/e072_prep_df.pkl", "rb") as fin:
    prep_df = pickle.load(fin)

In [14]:
with open("../data/preprocessed/e072_answer_pred_text_df .pkl", "rb") as fin:
    answer_pred_text_df = pickle.load(fin)

In [15]:
answer_pred_text_df["is_original"] = ~answer_pred_text_df.id.str.contains("^mlqa|^xquad")

In [69]:
import re
from string import punctuation

def baseline_kernel1_text_postprocess(answer_text: str, context: str) -> str:
    bad_starts = [".", ",", "(", ")", "-", "–", ",", ";"]
    bad_endings = ["...", "-", "(", ")", "–", ",", ";"]

    tamil_ad = "கி.பி"
    tamil_bc = "கி.மு"
    tamil_km = "கி.மீ"
    hindi_ad = "ई"
    hindi_bc = "ई.पू"

    if answer_text == "":
        return answer_text
    while any([answer_text.startswith(y) for y in bad_starts]):
        answer_text = answer_text[1:]
    while any([answer_text.endswith(y) for y in bad_endings]):
        if answer_text.endswith("..."):
            answer_text = answer_text[:-3]
        else:
            answer_text = answer_text[:-1]
    if answer_text.endswith("..."):
        answer_text = answer_text[:-3]
    if (
        any(
            [
                answer_text.endswith(tamil_ad),
                answer_text.endswith(tamil_bc),
                answer_text.endswith(tamil_km),
                answer_text.endswith(hindi_ad),
                answer_text.endswith(hindi_bc),
            ]
        )
        and answer_text + "." in context
    ):
        answer_text = answer_text + "."
    return answer_text


def baseline_kernel2_text_postprocess(answer_text: str, context: str) -> str:
    answer_text = baseline_kernel1_text_postprocess(
        answer_text=answer_text, context=context
    )
    if re.search("[0-9]\.$", answer_text):
        answer_text = answer_text[:-1]
    return answer_text


def baseline_kernel3_text_postprocess(answer_text: str, context: str) -> str:
    answer_text = " ".join(answer_text.split())
    answer_text = answer_text.strip("".join(list(filter(lambda x: x != "%", punctuation))))
    # answer_text = answer_text.strip('!"#$&\'()*+,-/:;<=>?@[\\]^_`{|}~')
    answer_text = baseline_kernel1_text_postprocess(
        answer_text=answer_text, context=context
    )
    return answer_text

def marathi_number_to_arabic_number(answer_text: str, context: str) -> str:
    if not answer_text.isnumeric():
        return answer_text

    marathi_to_arabic = {
        "०": "0",
        "१": "1",
        "२": "2",
        "३": "3",
        "४": "4",
        "५": "5",
        "६": "6",
        "७": "7",
        "८": "8",
        "९": "9",
    }
    
    new_answer_text = answer_text
    for marathi_num, arabic_num in marathi_to_arabic.items():
        new_answer_text = new_answer_text.replace(marathi_num, arabic_num)
    if context.find(new_answer_text) > 0:
        return new_answer_text
    return answer_text


def _map_marathi_to_arabic(text: str) -> str:
    marathi_to_arabic = {
        "०": "0",
        "१": "1",
        "२": "2",
        "३": "3",
        "४": "4",
        "५": "5",
        "६": "6",
        "७": "7",
        "८": "8",
        "९": "9",
    }
    new_text = text
    for marathi_num, arabic_num in marathi_to_arabic.items():
        new_text = new_text.replace(marathi_num, arabic_num)        
    return new_text

def _map_arabic_to_marathi(text: str) -> str:
    arabic_to_marathi = {
        "0": "०",
        "1": "१",
        "2": "२",
        "3": "३",
        "4": "४",
        "5": "५",
        "6": "६",
        "7": "७",
        "8": "८",
        "9": "९",
    }
    new_text = text
    for arabic_num, marathi_num in arabic_to_marathi.items():
        new_text = new_text.replace(arabic_num, marathi_num)
    return new_text

def bit_all_marathi_number_to_arabic_number(answer_text: str, context: str) -> str:
    # if answer_text.isnumeric():
    #     return answer_text
    
    min_find_index = 1000000000
    res_answer_text = answer_text

    answer_text_parts = answer_text.split()
    if len(answer_text_parts) > 12:
        return answer_text
    for i in range(2**len(answer_text_parts)):
        new_answer_text = answer_text
        for j in range(len(answer_text_parts)):
            if i & (1 << j):
                new_answer_text = new_answer_text.replace(answer_text_parts[j], _map_marathi_to_arabic(answer_text_parts[j]))
            else:
                new_answer_text = new_answer_text.replace(answer_text_parts[j], _map_arabic_to_marathi(answer_text_parts[j]))
        find_index = context.find(new_answer_text)
        if find_index > 0 and find_index < min_find_index:
            min_find_index = find_index
            res_answer_text = new_answer_text
    return res_answer_text


def text_postprocess(answer_pred, context):
    postprocessed_text = baseline_kernel1_text_postprocess(answer_pred, context)
    # postprocessed_text = baseline_kernel2_text_postprocess(postprocessed_text, context)
    postprocessed_text = baseline_kernel3_text_postprocess(postprocessed_text, context)
    # postprocessed_text = baseline_kernel3_text_postprocess(answer_pred, context)
    # postprocessed_text = baseline_kernel1_text_postprocess(postprocessed_text, context)
    postprocessed_text = marathi_number_to_arabic_number(postprocessed_text, context)
    ##### postprocessed_text = bit_all_marathi_number_to_arabic_number(postprocessed_text, context)
    return postprocessed_text

In [17]:
import Levenshtein

postprocessed_answer_pred = [text_postprocess(row["answer_pred"], row["context"]) for _, row in answer_pred_text_df.iterrows()]
answer_pred_text_df["new_answer_pred"] = postprocessed_answer_pred

new_jaccards = [jaccard(row["new_answer_pred"], row["answer_text"]) for _, row in answer_pred_text_df.iterrows()]
new_levenshteins = [Levenshtein.distance(" ".join(row["new_answer_pred"].split()), " ".join(row["answer_text"].split())) for _, row in answer_pred_text_df.iterrows()]

answer_pred_text_df["new_jaccard"] = new_jaccards
answer_pred_text_df["new_levenshtein"] = new_levenshteins

old_jaccard_mean = answer_pred_text_df.query("is_original").jaccard.mean()
old_jaccard_std = answer_pred_text_df.query("is_original").jaccard.std()
new_jaccard_mean = answer_pred_text_df.query("is_original").new_jaccard.mean()
new_jaccard_std = answer_pred_text_df.query("is_original").new_jaccard.std()
print(f"{old_jaccard_mean:.5f}+-{old_jaccard_std:.3f}   =>   {new_jaccard_mean:.5f}+-{new_jaccard_std:.3f}")

0.69184+-0.414   =>   0.70929+-0.407


In [66]:
import Levenshtein

postprocessed_answer_pred = [text_postprocess(row["answer_pred"], row["context"]) for _, row in answer_pred_text_df.iterrows()]
answer_pred_text_df["new_answer_pred"] = postprocessed_answer_pred

new_jaccards = [jaccard(row["new_answer_pred"], row["answer_text"]) for _, row in answer_pred_text_df.iterrows()]
new_levenshteins = [Levenshtein.distance(" ".join(row["new_answer_pred"].split()), " ".join(row["answer_text"].split())) for _, row in answer_pred_text_df.iterrows()]

answer_pred_text_df["new_jaccard"] = new_jaccards
answer_pred_text_df["new_levenshtein"] = new_levenshteins

old_jaccard_mean = answer_pred_text_df.query("is_original").jaccard.mean()
old_jaccard_std = answer_pred_text_df.query("is_original").jaccard.std()
new_jaccard_mean = answer_pred_text_df.query("is_original").new_jaccard.mean()
new_jaccard_std = answer_pred_text_df.query("is_original").new_jaccard.std()
print(f"{old_jaccard_mean:.5f}+-{old_jaccard_std:.3f}   =>   {new_jaccard_mean:.5f}+-{new_jaccard_std:.3f}")

0.69184+-0.414   =>   0.71001+-0.407


In [53]:
origin_answer_pred_text_df = answer_pred_text_df.query("is_original")
origin_answer_pred_text_df["jaccard_minus_new_jaccard"] = origin_answer_pred_text_df["jaccard"] - origin_answer_pred_text_df["new_jaccard"]
display(origin_answer_pred_text_df.sort_values(["jaccard_minus_new_jaccard", "id"]).head(20), origin_answer_pred_text_df.sort_values("jaccard_minus_new_jaccard").tail(20))

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,id,answer_pred,answer_text,jaccard,levenshtein,question,language,context,is_original,new_answer_pred,new_jaccard,new_levenshtein,jaccard_minus_new_jaccard
3097,076ee4006,२०१४,2014,0.000000,4,उत्तरी हिंद महासागर में बना हुदहुद तूफ़ान किस व...,hindi,हुदहुद एक उष्णकटिबंधीय चक्रवाती तूफान है। यह उ...,True,2014,1.0,0,-1.000000
1567,1b8635229,१४,14,0.000000,2,देवनागरी लिपि में कितने स्वर है?,hindi,देवनागरी एक भारतीय लिपि है जिसमें अनेक भारतीय ...,True,14,1.0,0,-1.000000
3116,2564a11bd,१६७४,1674,0.000000,4,मराठा साम्राज्य को किस साल में स्थापित किया गय...,hindi,मराठा साम्राज्य या मराठा महासंघ एक भारतीय साम्...,True,1674,1.0,0,-1.000000
3128,2af249652,18.,18,0.000000,1,आर्गन की परमाणु संख्या कितनी है?,hindi,आर्गन एक रासायनिक तत्व है। यह एक निष्क्रिय गैस...,True,18,1.0,0,-1.000000
1591,33d679522,६४,64,0.000000,2,शतरंज बोर्ड में कितने वर्ग होते हैं?,hindi,शतरंज (चैस) दो खिलाड़ियों के बीच खेला जाने वाल...,True,64,1.0,0,-1.000000
1629,5f7c6e0a7,-1799),1799,0.000000,2,फ्रांसीसी क्रांति किस वर्ष में समाप्त हुई थी?,hindi,फ्रांसीसी क्रांति (फ्रेंच: Révolution français...,True,1799,1.0,0,-1.000000
4738,73ff5d44e,१८,18,0.000000,2,महाभारत का युद्ध कितने दिनों तक चला था?,hindi,कुरुक्षेत्र युद्ध कौरवों और पाण्डवों के मध्य क...,True,18,1.0,0,-1.000000
1662,7ecaa9501,1852),1852,0.000000,1,फ़्रीड्रिक विलियम अगस्त फ्रोबेल की मृत्यु किस ...,hindi,फ़्रीड्रिक विलियम अगस्त फ्रोबेल (Friedrich Wil...,True,1852,1.0,0,-1.000000
4750,839bdc1fc,१९४५,1945,0.000000,4,संयुक्त राष्ट्र संगठन किस साल में स्थापित हुआ था?,hindi,"संयुक्त राष्ट्र () एक अंतरराष्ट्रीय संगठन है, ...",True,1945,1.0,0,-1.000000
6313,9aaa1672a,(1926,1926,0.000000,1,गुरु केलुचरण महापात्र का जन्म कब हुआ था?,hindi,गुरु केलुचरण महापात्र (1926-2004) ने ओड़िसीका ...,True,1926,1.0,0,-1.000000


,id,answer_pred,answer_text,jaccard,levenshtein,question,language,context,is_original,new_answer_pred,new_jaccard,new_levenshtein,jaccard_minus_new_jaccard
1691,a4e9767b3,\n1347 ईसवी,1327 से 1424 के बीच,0.000000,14,गुलबर्ग किला किस सदी में बनाया गया था?,hindi,गुलबर्ग किला उत्तर कर्नाटक के गुलबर्ग जिले में...,True,1347 ईसवी,0.000000,14,0.000000
1690,a3068dd53,९ अगस्त १९४२,९ अगस्त १९४२,1.000000,0,भारत छोड़ो आंदोलन कब शुरू हुआ था?,hindi,"भारत छोड़ो आन्दोलन, द्वितीय विश्वयुद्ध के समय ...",True,९ अगस्त १९४२,1.000000,0,0.000000
1693,a8d2f1ef6,ईक्वस कैबेलस,ईक्वस कैबेलस,1.000000,0,घोड़े का वैज्ञानिक नाम क्या है?,hindi,घोड़ा या अश्व (Equus ferus caballus; ऐक़्वस फ़...,True,ईक्वस कैबेलस,1.000000,0,0.000000
1706,b02a8d62f,१४ सितम्बर,१४ सितम्बर,1.000000,0,भारत में किस दिन को हिंदी दिवस के रूप में मनाय...,hindi,हिन्दी दिवस प्रत्येक वर्ष १४ सितम्बर को मनाया ...,True,१४ सितम्बर,1.000000,0,0.000000
1705,af30436d6,लास वेगास,लास वेगास,1.000000,0,आन्द्रे अगासी का जन्म किस शहर में हुआ था?,hindi,आन्द्रे अगासी का जन्म 29 अप्रैल 1970 अमेरिका द...,True,लास वेगास,1.000000,0,0.000000
1704,af2573e66,भारत,भारत,1.000000,0,किस देश ने शतरंज के गेम का अविष्कार किया था?,hindi,शतरंज (चैस) दो खिलाड़ियों के बीच खेला जाने वाल...,True,भारत,1.000000,0,0.000000
1703,ade00ec7a,1945,1945,1.000000,0,कब संयुक्त राज्य अमेरिका ने जापान (हिरोशिमा) म...,hindi,जापान के प्राचीन इतिहास के संबंध में कोई निश्च...,True,1945,1.000000,0,0.000000
1702,ad90053ce,"ஜனவரி 10, 2008",2009 ஆம் ஆண்டு மார்ச் 23,0.000000,22,டாடா நானோ கார் எப்போது இந்தியாவில் விற்பனைக்கு...,tamil,டாடா நானோ என்பது டாடா மோட்டார்ஸ் நிறுவனத்தால் ...,True,"ஜனவரி 10, 2008",0.000000,22,0.000000
1692,a52d546ba,கற்கருவிகளைப்,கற்கருவிகளைப்,1.000000,0,ஆதி மனிதனின் வேட்டையாட பயன்படுத்திய ஆயுதம் எது?,tamil,பூமியில் வாழும் பாலூட்டிகளில் மிக மிக முன்னேற்...,True,கற்கருவிகளைப்,1.000000,0,0.000000
1701,acc5fc295,१८४८,२९ अप्रैल १८४८,0.333333,10,राजा रवि वर्मा का जन्म कब हुआ?,hindi,राजा रवि वर्मा (१८४८ - १९०६) भारत के विख्यात च...,True,१८४८,0.333333,10,0.000000


In [44]:
origin_answer_pred_text_df.query("language == 'hindi'").sort_values("jaccard").head(150)

,id,answer_pred,answer_text,jaccard,levenshtein,question,language,context,is_original,new_answer_pred,new_jaccard,new_levenshtein,jaccard_minus_new_jaccard
4851,ebe48c5e6,\n1981,"२ जुलाई, १९८१",0.000000,13,इंफोसिस लिमिटेड की स्थापना किस वर्ष में हुई थी?,hindi,"इन्फोसिस लिमिटेड (BSE:, Nasdaq:) एक बहुराष्ट्र...",True,1981,0.000000,13,0.000000
6232,35ed7e197,1953,2003,0.000000,3,एनिमेटेड फिल्म ''पीटर पैन'' किस वर्ष रिलीज़ की ...,hindi,पीटर पैन स्कॉटिश उपन्यासकार एवं नाटककार जे.एम....,True,1953,0.000000,3,0.000000
6225,311617604,मध्य अमेरिका,"""मध्य अमेरिका""",0.000000,2,मेसोअमेरिका' का यूनानी भाषा में शाब्दिक अर्थ क...,hindi,मेसोअमेरिका या मेसो-अमेरिका (Spanish: Mesoamér...,True,मध्य अमेरिका,0.000000,2,0.000000
1591,33d679522,६४,64,0.000000,2,शतरंज बोर्ड में कितने वर्ग होते हैं?,hindi,शतरंज (चैस) दो खिलाड़ियों के बीच खेला जाने वाल...,True,64,1.000000,0,-1.000000
6221,284fb5c58,जहांगीर,अकबर,0.000000,6,उत्तर प्रदेश के सिकन्दरा में स्थित अकबर का मकब...,hindi,सिकंदरा आगरा से चार किलोमीटर की दूरी पर स्थित ...,True,जहांगीर,0.000000,6,0.000000
1599,3d472c115,मुग़लसराय,पण्डित दीनदयाल उपाध्याय जंक्शन रेलवे स्टेशन,0.000000,40,एशिया का सबसे बड़ा रेलवे यार्ड कौन सा है?,hindi,पण्डित दीनदयाल उपाध्याय जंक्शन रेलवे स्टेशन (प...,True,मुग़लसराय,0.000000,40,0.000000
6217,2625178c9,लोरेन पॉवेल,लौरेन पावेल,0.000000,2,स्टीव जॉब्स की पत्नी का नाम क्या था?,hindi,स्टीव जॉब्स के विचार हिंदी में [1]\nस्टीवन पॉल...,True,लोरेन पॉवेल,0.000000,2,0.000000
1607,49dffdf0f,1591),1591 ई.,0.000000,3,चारमीनार का निर्माण किस वर्ष में हुआ था?,hindi,Coordinates: \n\nचारमीनार (उर्दू: چار مینار) 1...,True,1591,0.500000,3,-0.500000
1610,4ca677224,विद्युत-मापी,kWh,0.000000,12,ऊर्जा को किस इकाई द्वारा मापा जाता है?,hindi,विद्युत-मापी (Electricity meters) या 'ऊर्जामाप...,True,विद्युत-मापी,0.000000,12,0.000000
3229,9658a986b,"1,484km2","१,४८४ वर्ग किलोमीटर",0.000000,18,दिल्ली का क्षेत्रफल कितना है?,hindi,दिल्ली समुद्रतल से ७०० से १००० फीट की ऊँचाई पर...,True,"1,484km2",0.000000,18,0.000000


In [54]:
origin_answer_pred_text_df.query("language == 'hindi'").sort_values("new_jaccard").head(150)

,id,answer_pred,answer_text,jaccard,levenshtein,question,language,context,is_original,new_answer_pred,new_jaccard,new_levenshtein,jaccard_minus_new_jaccard
1671,87c08ced4,सुकेंद्रिक,युकेरियोटी,0.000000,7,जीवविज्ञान में सुकेंद्रिक कोशिकाओं वाले जीवों ...,hindi,सुकेंद्रिक या युकेरियोट (eukaryote) एक जीव को ...,True,सुकेंद्रिक,0.000000,7,0.000000
1574,23dbf4302,1832,1769,0.000000,3,सबसे पहली कार का आविष्कार कब हुआ था?,hindi,"गाड़ी, मोटरवाहन , कार, मोटरकार या ऑटोमोबाइल एक...",True,1832,0.000000,3,0.000000
6236,3a4db1dda,ventriloquism,ventriloquism',0.000000,1,बोलती कठपुतली कला' को अंग्रेज़ी भाषा में किस ना...,hindi,बोलती कठपुतली कला एक मंच-कला है जिसमें एक व्...,True,ventriloquism,0.000000,1,0.000000
6232,35ed7e197,1953,2003,0.000000,3,एनिमेटेड फिल्म ''पीटर पैन'' किस वर्ष रिलीज़ की ...,hindi,पीटर पैन स्कॉटिश उपन्यासकार एवं नाटककार जे.एम....,True,1953,0.000000,3,0.000000
1581,2b41f3744,\nमुंबई,मुम्बई,0.000000,2,"भारत का हिन्दी चलचित्र एवं दूरदर्शन उद्योग, कि...",hindi,भारत के पश्चिमी तट पर स्थित मुंंबई (पूर्व नाम ...,True,मुंबई,0.000000,2,0.000000
6225,311617604,मध्य अमेरिका,"""मध्य अमेरिका""",0.000000,2,मेसोअमेरिका' का यूनानी भाषा में शाब्दिक अर्थ क...,hindi,मेसोअमेरिका या मेसो-अमेरिका (Spanish: Mesoamér...,True,मध्य अमेरिका,0.000000,2,0.000000
6221,284fb5c58,जहांगीर,अकबर,0.000000,6,उत्तर प्रदेश के सिकन्दरा में स्थित अकबर का मकब...,hindi,सिकंदरा आगरा से चार किलोमीटर की दूरी पर स्थित ...,True,जहांगीर,0.000000,6,0.000000
6217,2625178c9,लोरेन पॉवेल,लौरेन पावेल,0.000000,2,स्टीव जॉब्स की पत्नी का नाम क्या था?,hindi,स्टीव जॉब्स के विचार हिंदी में [1]\nस्टीवन पॉल...,True,लोरेन पॉवेल,0.000000,2,0.000000
4704,4f41fec88,विल्हेम रॉक्स,रॉस ग्रेनविले हैरिसन,0.000000,16,कोशिका संवर्धन का आविष्कार किसने किया?,hindi,कोशिका संवर्धन एक ऐसी जटिल प्रक्रिया है जिसके ...,True,विल्हेम रॉक्स,0.000000,16,0.000000
3208,7f6531589,राजा भोज,भोजदेव,0.000000,7,भोजेश्वर मन्दिर का निर्माण किसने कराया?,hindi,भोजेश्वर मन्दिर मध्य प्रदेश की राजधानी भोपाल स...,True,राजा भोज,0.000000,7,0.000000


In [15]:
origin_answer_pred_text_df.query("language == 'tamil'").sort_values("jaccard").head(150)

,id,answer_pred,answer_text,jaccard,levenshtein,question,language,context,is_original,new_answer_pred,new_jaccard,new_levenshtein,jaccard_minus_new_jaccard
6391,faad03372,"துண்டித்தல் பயாப்ஸி, இதில் முழு கட்டியும் அகற...",உடல் திசு ஆய்வு,0.000000,43,பயாப்ஸி என்றால் என்ன?,tamil,மார்பகப் புற்றுநோய் அல்லது மார்புப் புற்று நோய...,True,"துண்டித்தல் பயாப்ஸி, இதில் முழு கட்டியும் அகற்...",0.000000,43,0.000000
4686,3e9696e57,கடற்பாசிகள்,விலங்கு,0.000000,10,மெல்லுடலிகள் எந்த திணையை சார்ந்தது?,tamil,"விலங்குகள் (Animals), அனிமாலியா (Animalia) அல்...",True,கடற்பாசிகள்,0.000000,10,0.000000
4701,4b69614bc,பைலேடரியா,நெமடோடா,0.000000,7,உருளைப்புழு எந்த தொகுதியை சேர்ந்தது?,tamil,"விலங்குகள் (Animals), அனிமாலியா (Animalia) அல்...",True,பைலேடரியா,0.000000,7,0.000000
1561,11d635808,நிலவே என்னிடம் நெருங்காதே,அத்தானோடு இப்படியிருந்து ௭த்தனை நாளாச்சு,0.000000,33,எஸ். பி. பாலசுப்பிரமணியத்தின் முதல் பாடல் எது?,tamil,ஸ்ரீபதி பண்டிதாரத்யுல பாலசுப்ரமணியம் (பிறப்பு ...,True,நிலவே என்னிடம் நெருங்காதே,0.000000,33,0.000000
1560,112d5df19,1010,1000,0.000000,1,பிரிஹதீஸ்வரர் கோயில் எவ்வளவு வயது?,tamil,"தஞ்சைப் பிரகதீசுவரர் கோயில் என்றும், தஞ்சைப் ப...",True,1010,0.000000,1,0.000000
4707,5414397bd,பானாஜியோடிஸ் ஸௌட்ஸாஸ்,பிரான்ஸ்,0.000000,16,முதல் குளிர்கால ஒலிம்பிக் விளையாட்டு போட்டி யா...,tamil,ஒலிம்பிக் விளையாட்டுக்கள் (Olympic Games அல்லத...,True,பானாஜியோடிஸ் ஸௌட்ஸாஸ்,0.000000,16,0.000000
4710,56021f924,1850,"சனவரி 28, 1892",0.000000,12,தொலைபேசி எந்த ஆண்டு இந்தியாவில் அறிமுகமானது?,tamil,இந்தியாவில் தொலைத் தொடர்புவருமானம் (மொத்தம்) $...,True,1850,0.000000,12,0.000000
1547,0115b1c86,Thiomargarita namibiensis,களிறு,0.000000,25,பூமியில் மிகப்பெரிய பாலூட்டி எது?,tamil,"மிகப்பெரிய உயிரினங்கள் கனவளவு, திணிவு, உயரம், ...",True,Thiomargarita namibiensis,0.000000,25,0.000000
4740,77a263d51,என்ரி பெக்கரல்,ஹென்றி பெக்கொரெலு,0.000000,6,கதிரியக்கத்தை கண்டுபிடித்தவர் யார்?,tamil,"கதிரியக்கம் (radioactivity, radioactive decay,...",True,என்ரி பெக்கரல்,0.000000,6,0.000000
219,fd235557c,செவ்வரத்தை,Hibiscus rosa-sinensis,0.000000,22,செம்பருத்தியின் உயிரியலில் பெயர் என்ன?,tamil,செம்பருத்தி அல்லது செவ்வரத்தை (Hibiscus rosa-s...,True,செவ்வரத்தை,0.000000,22,0.000000


In [19]:
# bad ends?
# ஏரி = lake
# 
origin_answer_pred_text_df[origin_answer_pred_text_df.answer_pred.str.contains("மீட்டர்")]

,id,answer_pred,answer_text,jaccard,levenshtein,question,language,context,is_original,new_answer_pred,new_jaccard,new_levenshtein,jaccard_minus_new_jaccard
46,34ed796e6,"329,847 சதுர கிலோமீட்டர்கள்","329,847 சதுர கிலோமீட்டர்கள்",1.000000,0,மலேசியா நாட்டின் பரப்பளவு என்ன?,tamil,"மலேசியா (Malaysia), தென்கிழக்காசியாவில் உள்ள ஒ...",True,"329,847 சதுர கிலோமீட்டர்கள்",1.000000,0,0.0
66,4fee1f0f8,86.928 சதுர கிலோ மீட்டர்கள்,86.928,0.250000,21,பாரிஸ் நகரத்தின் பரப்பளவு என்ன?,tamil,பாரிஸ் அல்லது பாரி எனப்படுவது பிரான்ஸ் நாட்டின...,True,86.928 சதுர கிலோ மீட்டர்கள்,0.250000,21,0.0
73,5502c73af,இரண்டு மில்லியன் சதுர கிலோமீட்டர்,இரண்டு மில்லியன் சதுர கிலோமீட்டர்,1.000000,0,மெக்சிகோ நாட்டின் பரப்பளவு என்ன?,tamil,"மெக்சிக்கோ (எசுப்பானியம்: México, ""மெஹிக்கோ"") ...",True,இரண்டு மில்லியன் சதுர கிலோமீட்டர்,1.000000,0,0.0
148,ae3fbd19f,"329,847 சதுர கிலோமீட்டர்கள்","329,847",0.333333,20,மலேசியா நாட்டின் பரப்பளவு என்ன?,tamil,"மலேசியா (Malaysia), தென்கிழக்காசியாவில் உள்ள ஒ...",True,"329,847 சதுர கிலோமீட்டர்கள்",0.333333,20,0.0
177,c5fdb266d,"1,483 சதுர கிலோ மீட்டர்","1,484 ச.கிமீ",0.000000,13,தில்லி மாநகரத்தின் பரப்பளவு என்ன?,tamil,"தில்லி அல்லது டெல்லி (Delhi, இந்தி: दिल्ली, பஞ...",True,"1,483 சதுர கிலோ மீட்டர்",0.000000,13,0.0
3114,221526dfb,"\n38,852 சதுர கிலோ மீட்டர்","38,852",0.250000,18,கேரள மாநிலத்தின் பரப்பளவு என்ன?,tamil,"{{IPA}}, formats symbols of the International ...",True,"38,852 சதுர கிலோ மீட்டர்",0.250000,18,0.0
3148,3fe627e15,"4,694 மீட்டர்","10,911",0.000000,12,பசிபிக் பெருங்கடல்லின் அதிகபட்ச ஆழம் எவ்வளவு?,tamil,"கடல் (ஒலிப்பு) அல்லது ஆழி(Sea), உலகப் பெருங்க...",True,"4,694 மீட்டர்",0.000000,12,0.0
3245,ac51a153d,36 முதல் 42 மீட்டர்,42,0.250000,17,பனை மரத்தின் சராசரி உயரம் என்ன?,tamil,"பனை (Palmyra Palm), புல்லினத்தைச் சேர்ந்த ஒரு ...",True,36 முதல் 42 மீட்டர்,0.250000,17,0.0
4722,65fec9728,"16,579 சதுர கிலோ மீட்டர்","16,579 சதுர கிலோ மீட்டர்",1.000000,0,நாகாலாந்து மாநிலத்தின் பரப்பளவு என்ன?,tamil,நாகாலாந்து (Nagaland) இந்தியாவின் வடகிழக்குப் ...,True,"16,579 சதுர கிலோ மீட்டர்",1.000000,0,0.0
4863,fc36a6c01,"1,376 கிலோ மீட்டர்","1,376 கிலோ மீட்டர்",1.000000,0,யமுனை ஆற்றின் நீளம் என்ன?,tamil,யமுனை ஆறு\nவட இந்தியாவின் முக்கியமான ஆறுகளுள் ...,True,"1,376 கிலோ மீட்டர்",1.000000,0,0.0


In [46]:
row = origin_answer_pred_text_df.query("id == 'b766260e6'").iloc[0]

row["context"].find("15 जून 1902"), row["context"].find("१५ जून १९०२")

(219, 39)

## 空白とかでズレてる？

In [11]:
origin_answer_pred_text_df.query("question == 'ரேடியம் எப்போது கண்டுபிடிக்கப்பட்டது?'")

,id,answer_pred,answer_text,jaccard,levenshtein,question,language,context,is_original,new_answer_pred,new_jaccard,new_levenshtein,jaccard_minus_new_jaccard
68,5179ed725,1898,1898,1.0,0,ரேடியம் எப்போது கண்டுபிடிக்கப்பட்டது?,tamil,ரேடியம் (Radium) என்பது Ra என்ற மூலக்கூற்று வா...,True,1898,1.0,0,0.0
165,bc9f0d533,1898,மேரிகியூரி மற்றும் பியரிகியூரி,0.0,30,ரேடியம் எப்போது கண்டுபிடிக்கப்பட்டது?,tamil,ரேடியம் (Radium) என்பது Ra என்ற மூலக்கூற்று வா...,True,1898,0.0,30,0.0


In [ ]:
origin_answer_pred_text_df.loc[68]["context"] == origin_answer_pred_text_df.loc[165]["context"]

In [13]:
origin_answer_pred_text_df.loc[68]["context"]

'ரேடியம் (Radium) என்பது Ra என்ற மூலக்கூற்று வாய்ப்பாடு கொண்ட  கதிர்வீச்சு இயல்புள்ள ஒரு தனிமமாகும். இதன் அணு எண் 88 ஆகும். இதன் அணுநிறை 226 ஆகும். தனிம வரிசை அட்டவணையின்  நெடுங்குழு 2 இல் இடம்பெற்றுள்ள ஆறாவது தனிமம் ரேடியம் ஆகும். காரமண் உலோகம் என்றும் இதை வகைப்படுத்துவர். தூய்மையான ரேடியம் வெள்ளியைப் போல வெண்மை நிறமுடையதாக உள்ளது. ஆனால் காற்றில் வெளிப்பட நேர்ந்தால் ரேடியம் ஆக்சிசனுக்குப் பதிலாக நைட்ரசனுடன் உடனடியாக வினைபுரிந்து ரேடியம் நைட்ரைடு (Ra3N2) என்ற கருப்பு நிற மேற்பரப்பு அடுக்காக உருவாகிறது. ரேடியத்தின் அனைத்து ஐசோடோப்புகளும் அதிகமான கதிரியக்கத் தன்மையுடையனவாகும். இவற்றில் ரேடியம் -226 என்ற ஐசோடோப்பு அதிக நிலைப்புத்தன்மை கொண்டதாக உள்ளது. இதனுடைய அரை ஆயுள் காலம் 1600 ஆண்டுகளாகும். கதிரியக்கச் சிதைவடைந்து இந்த ஐசோடோப்பு ரேடான் வாயுவாக, குறிப்பாக ரேடான் - 222 என்ற ஐசோடோப்பாக மாறுகிறது.  ரேடியம் சிதைவடையும்போது அயனியாக்கும் கதிர் ஒரு விளைபொருளாகும். இது ஒளிரும் வேதிப்பொருட்களை கிளர்வூட்டி கதிரியக்க ஒளிர்வைத் தருகிறது.\nரேடியம் குளோரைடு வடிவத்தில் ரேடியம்  1898 ஆம் ஆண்டு மேரிகியூ

In [14]:
origin_answer_pred_text_df.loc[165]["context"]

'ரேடியம் (Radium) என்பது Ra என்ற மூலக்கூற்று வாய்ப்பாடு கொண்ட  கதிர்வீச்சு இயல்புள்ள ஒரு தனிமமாகும். இதன் அணு எண் 88 ஆகும். இதன் அணுநிறை 226 ஆகும். தனிம வரிசை அட்டவணையின்  நெடுங்குழு 2 இல் இடம்பெற்றுள்ள ஆறாவது தனிமம் ரேடியம் ஆகும். காரமண் உலோகம் என்றும் இதை வகைப்படுத்துவர். தூய்மையான ரேடியம் வெள்ளியைப் போல வெண்மை நிறமுடையதாக உள்ளது. ஆனால் காற்றில் வெளிப்பட நேர்ந்தால் ரேடியம் ஆக்சிசனுக்குப் பதிலாக நைட்ரசனுடன் உடனடியாக வினைபுரிந்து ரேடியம் நைட்ரைடு (Ra3N2) என்ற கருப்பு நிற மேற்பரப்பு அடுக்காக உருவாகிறது. ரேடியத்தின் அனைத்து ஐசோடோப்புகளும் அதிகமான கதிரியக்கத் தன்மையுடையனவாகும். இவற்றில் ரேடியம் -226 என்ற ஐசோடோப்பு அதிக நிலைப்புத்தன்மை கொண்டதாக உள்ளது. இதனுடைய அரை ஆயுள் காலம் 1600 ஆண்டுகளாகும். கதிரியக்கச் சிதைவடைந்து இந்த ஐசோடோப்பு ரேடான் வாயுவாக, குறிப்பாக ரேடான் - 222 என்ற ஐசோடோப்பாக மாறுகிறது.  ரேடியம் சிதைவடையும்போது அயனியாக்கும் கதிர் ஒரு விளைபொருளாகும். இது ஒளிரும் வேதிப்பொருட்களை கிளர்வூட்டி கதிரியக்க ஒளிர்வைத் தருகிறது.\nரேடியம் குளோரைடு வடிவத்தில் ரேடியம்  1898 ஆம் ஆண்டு மேரிகியூ

In [16]:
"மேரிகியூரி மற்றும் பியரிகியூரி".split()

['மேரிகியூரி', 'மற்றும்', 'பியரிகியூரி']

In [30]:
ID = "33d679522"

prep_df.query(f"id == '{ID}'")

,id,context,question,answer_text,answer_start,language,top20_context,answer_text_count,input_ids,token_type_ids,attention_mask,sequence_ids,offset_mapping,overflowing_batch_id,duplicated_elems_num_with,part_answer_text_count,start_position,end_position,segmentation_position,is_contain_answer_text,start_logits,end_logits,char_index_score_dict,max_score,max_index,is_original
9687,33d679522,शतरंज (चैस) दो खिलाड़ियों के बीच खेला जाने वाल...,शतरंज बोर्ड में कितने वर्ग होते हैं?,64,1604,hindi,शतरंज (चैस) दो खिलाड,2,"[0, 8933, 21100, 36860, 51028, 421, 180513, 40...","[None, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, None, Non...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[(-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, ...",0,-1,0,0,0,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0,"[2.676165819168091, -8.631897926330566, -10.05...","[0.4703912138938904, -12.662092208862305, -12....",{},-1.000000e+08,-100000000,True
9688,33d679522,शतरंज (चैस) दो खिलाड़ियों के बीच खेला जाने वाल...,शतरंज बोर्ड में कितने वर्ग होते हैं?,64,1604,hindi,शतरंज (चैस) दो खिलाड,2,"[0, 8933, 21100, 36860, 51028, 421, 180513, 40...","[None, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, None, Non...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[(-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, ...",1,-1,1,240,240,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1,"[4.357115745544434, -9.183385848999023, -10.01...","[2.5425636768341064, -12.545413970947266, -12....","{""1604"": 3.2530715465545654}",3.253072e+00,1604,True
9689,33d679522,शतरंज (चैस) दो खिलाड़ियों के बीच खेला जाने वाल...,शतरंज बोर्ड में कितने वर्ग होते हैं?,64,1604,hindi,शतरंज (चैस) दो खिलाड,2,"[0, 8933, 21100, 36860, 51028, 421, 180513, 40...","[None, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, None, Non...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[(-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, ...",2,-1,0,0,0,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0,"[4.625796318054199, -8.036948204040527, -9.685...","[2.9389963150024414, -12.166171073913574, -11....",{},-1.000000e+08,-100000000,True
9690,33d679522,शतरंज (चैस) दो खिलाड़ियों के बीच खेला जाने वाल...,शतरंज बोर्ड में कितने वर्ग होते हैं?,64,1604,hindi,शतरंज (चैस) दो खिलाड,2,"[0, 8933, 21100, 36860, 51028, 421, 180513, 40...","[None, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, None, Non...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[(-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, ...",3,-1,0,0,0,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0,"[4.742928981781006, -7.034996032714844, -9.389...","[2.9818475246429443, -11.735984802246094, -11....",{},-1.000000e+08,-100000000,True
9691,33d679522,शतरंज (चैस) दो खिलाड़ियों के बीच खेला जाने वाल...,शतरंज बोर्ड में कितने वर्ग होते हैं?,64,1604,hindi,शतरंज (चैस) दो खिलाड,2,"[0, 8933, 21100, 36860, 51028, 421, 180513, 40...","[None, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, None, Non...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[(-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, ...",4,-1,0,0,0,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0,"[4.842467784881592, -7.354754447937012, -9.612...","[3.032099485397339, -11.373685836791992, -11.9...",{},-1.000000e+08,-100000000,True
9692,33d679522,शतरंज (चैस) दो खिलाड़ियों के बीच खेला जाने वाल...,शतरंज बोर्ड में कितने वर्ग होते हैं?,64,1604,hindi,शतरंज (चैस) दो खिलाड,2,"[0, 8933, 21100, 36860, 51028, 421, 180513, 40...","[None, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, None, Non...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[(-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, ...",5,-1,0,0,0,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0,"[4.690090179443359, -7.5801920890808105, -9.59...","[2.966197967529297, -11.628213882446289, -11.8...",{},-1.000000e+08,-100000

In [36]:
row = prep_df.loc[9687]
print(row["is_contain_answer_text"])
print(row["question"])
print(row["answer_text"])
print(row["char_index_score_dict"])
print(row["offset_mapping"])
print(row["context"][0:1302])

0
शतरंज बोर्ड में कितने वर्ग होते हैं?
64
{}
[(-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, -1), (0, 1), (1, 3), (3, 5), (5, 7), (7, 8), (8, 9), (9, 10), (10, 11), (11, 14), (14, 25), (25, 28), (28, 32), (32, 36), (36, 37), (37, 42), (42, 47), (47, 50), (50, 56), (56, 58), (58, 62), (62, 66), (66, 67), (67, 69), (69, 70), (70, 74), (74, 77), (77, 78), (78, 83), (83, 90), (90, 97), (97, 98), (98, 100), (100, 102), (102, 105), (105, 108), (108, 113), (113, 116), (116, 117), (117, 118), (118, 120), (120, 122), (122, 124), (124, 128), (128, 131), (131, 135), (135, 141), (141, 144), (144, 151), (151, 154), (154, 158), (158, 161), (161, 163), (163, 166), (166, 170), (170, 175), (175, 176), (176, 181), (181, 186), (186, 189), (189, 192), (192, 195), (195, 199), (199, 203), (203, 205), (205, 210), (210, 213), (213, 215), (215, 217), (217, 219), (219, 222), (222, 225), (225, 226), (226, 232), (232, 240), (240, 244), 